# Import necessary libraries

<b> Note that </b>: If you're using Jupyter Notebook, you wouldn't need the drive.mount("/content/drive").

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import gensim

Mounted at /content/drive


<b> Note that: </b> If you're using a local directory, you'll have to read_excel from the appropriate directory.

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/IS424 Depression Project/model_data_v2.xlsx")

In [ ]:
# This step is to drop unnecessary columns, and empty rows.

df = df.rename(columns={"text_cleaned": "text"})
df = df.drop("Unnamed: 0", axis=1)
df['text'] = df['text'].astype("str")
df = df.dropna()

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000

# Max number of words in each thread. For this one, we just used the max length of word of all sentences in the data.
MAX_SEQUENCE_LENGTH = max([len(s.split()) for s in df['text']])

# Hyperparameter. We tested with different parameters and 128 returns the best results.
EMBEDDING_DIM = 128

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

Found 53660 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = pd.get_dummies(df['depression']).values
print('Shape of label tensor:', Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 99)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

Shape of data tensor: (38362, 2521)
Shape of label tensor: (38362, 2)
(26853, 2521) (26853, 2)
(11509, 2521) (11509, 2)


In [ ]:
# This is for testing later. Y_test_arr stores the original classes' outputs in a 1D array.
Y_test_arr = []
for idx in range(len(Y_test)):
  if Y_test[idx][0] == 1:
    Y_test_arr.append(0)
  else:
    Y_test_arr.append(1) 

# LSTM + Keras' Word Embeddings


In [ ]:
|# LSTM
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
import tensorflow as tf 
from tensorflow import keras
from keras.regularizers import l2

model = Sequential()
adam = tf.optimizers.Adam(learning_rate=0.001)
model.add(layers.Embedding(len(word_index)+1, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(32, return_sequences=True, dropout=0.5, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
model.add(GlobalMaxPooling1D())
model.add(Dense(10,activation='relu', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))) 
model.add(layers.Dense(2, activation="softmax"))
model.compile(optimizer=adam, loss="binary_crossentropy", 
     metrics=['accuracy'])

epochs = 25
batch_size = 64

	
es = EarlyStopping(monitor='val_loss', mode="min", patience=3)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.3, callbacks=[es])

Epoch 1/25
294/294 [==============================] - 86s 283ms/step - loss: 0.3894 - accuracy: 0.9064 - val_loss: 0.2323 - val_accuracy: 0.9506
Epoch 2/25
294/294 [==============================] - 82s 280ms/step - loss: 0.1659 - accuracy: 0.9666 - val_loss: 0.1710 - val_accuracy: 0.9600
Epoch 3/25
294/294 [==============================] - 83s 283ms/step - loss: 0.1236 - accuracy: 0.9786 - val_loss: 0.1623 - val_accuracy: 0.9610
Epoch 4/25
294/294 [==============================] - 84s 285ms/step - loss: 0.1004 - accuracy: 0.9847 - val_loss: 0.1578 - val_accuracy: 0.9614
Epoch 5/25
294/294 [==============================] - 84s 285ms/step - loss: 0.0869 - accuracy: 0.9882 - val_loss: 0.1629 - val_accuracy: 0.9597
Epoch 6/25
294/294 [==============================] - 84s 284ms/step - loss: 0.0745 - accuracy: 0.9905 - val_loss: 0.1596 - val_accuracy: 0.9594
Epoch 7/25
294/294 [==============================] - 83s 284ms/step - loss: 0.0650 - accuracy: 0.9922 - val_loss: 0.1593 - val_ac

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

360/360 [==============================] - 42s 116ms/step - loss: 0.1578 - accuracy: 0.9615
Test set
  Loss: 0.158
  Accuracy: 0.962


In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

predictions = model.predict(X_test)
predictions = [np.argmax(prediction) for prediction in predictions]
precision, recall, f1_score, none = precision_recall_fscore_support(Y_test_arr, predictions, average='weighted')
print("Precision is: ", precision)
print("Recall is: ", recall)
print("F1 Score is: ", f1_score)

Precision is:  0.9615153440042628
Recall is:  0.9615083847423755
F1 Score is:  0.9615106347249784


# Bi-LSTM + Keras' Word Embeddings

In [ ]:
#Bi-LSTM

from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
import tensorflow as tf 
from tensorflow import keras
from keras.regularizers import l2
model = Sequential()
adam = tf.optimizers.Adam(learning_rate=0.001)
model.add(layers.Embedding(len(word_index)+1, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=0.5, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001))))
model.add(GlobalMaxPooling1D())
model.add(Dense(10,activation='relu', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))) 
model.add(layers.Dense(2, activation="softmax"))
model.compile(optimizer=adam, loss="binary_crossentropy", 
     metrics=['accuracy'])

epochs = 25
batch_size = 64

	
es = EarlyStopping(monitor='val_loss', mode="min", patience=3)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.3, callbacks=[es])

Epoch 1/25
294/294 [==============================] - 171s 569ms/step - loss: 0.3986 - accuracy: 0.9105 - val_loss: 0.2091 - val_accuracy: 0.9633
Epoch 2/25
294/294 [==============================] - 165s 562ms/step - loss: 0.1612 - accuracy: 0.9743 - val_loss: 0.1752 - val_accuracy: 0.9647
Epoch 3/25
294/294 [==============================] - 167s 568ms/step - loss: 0.1186 - accuracy: 0.9843 - val_loss: 0.1655 - val_accuracy: 0.9641
Epoch 4/25
294/294 [==============================] - 167s 570ms/step - loss: 0.0929 - accuracy: 0.9887 - val_loss: 0.1658 - val_accuracy: 0.9605
Epoch 5/25
294/294 [==============================] - 168s 571ms/step - loss: 0.0837 - accuracy: 0.9903 - val_loss: 0.1587 - val_accuracy: 0.9628
Epoch 6/25
294/294 [==============================] - 167s 569ms/step - loss: 0.0751 - accuracy: 0.9922 - val_loss: 0.1518 - val_accuracy: 0.9641
Epoch 7/25
294/294 [==============================] - 166s 566ms/step - loss: 0.0611 - accuracy: 0.9949 - val_loss: 0.1496 -

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

predictions = model.predict(X_test)
predictions = [np.argmax(prediction) for prediction in predictions]
precision, recall, f1_score, none = precision_recall_fscore_support(Y_test_arr, predictions, average='weighted')
print("Precision is: ", precision)
print("Recall is: ", recall)
print("F1 Score is: ", f1_score)

360/360 [==============================] - 85s 236ms/step - loss: 0.1608 - accuracy: 0.9579
Test set
  Loss: 0.161
  Accuracy: 0.958
Precision is:  0.9582008817589148
Recall is:  0.9579459553392997
F1 Score is:  0.9579097474929152


# LSTM + CBOW

- Choose where you want to save the embedding_word2vec txt files.
- You will be retrieving from that directory in the next code!

In [ ]:
"""
CBOW Model
"""
text_sentences = df['text'].apply(lambda x: x.split())

model = gensim.models.Word2Vec(sentences=text_sentences, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
words = list(model.wv.vocab)
print(len(words))

# Where you want to save.
filename = 'depression_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)


53756


In [ ]:
import numpy as np
embeddings_index = {}
f = open("/content/depression_embedding_word2vec.txt")
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddings_index[word] = coefs
f.close()

num_words = len(word_index) + 1
word2vec_embedding_matrix = np.zeros((num_words, 128))

for word, i in word_index.items():
  if i > num_words: 
    continue

  embedding_vector = embeddings_index.get(word) 
  if embedding_vector is not None:
    word2vec_embedding_matrix[i] = embedding_vector 
print(num_words)


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)

word_index = tokenizer.word_index
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)


Y = pd.get_dummies(df['depression']).values
print('Shape of label tensor:', Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 99)
# X_train, X_test, Y_train, Y_test = train_test_split(X_svd, Y, test_size = 0.30, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)



53661
Shape of label tensor: (38362, 2)
(26853, 2521) (26853, 2)
(11509, 2521) (11509, 2)


In [ ]:
# CBOW LSTM

from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
import tensorflow as tf 
from tensorflow import keras
from keras.regularizers import l2


model = Sequential()
adam = tf.optimizers.Adam(learning_rate=0.001)
model.add(Embedding(num_words, 128, weights=[word2vec_embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, 
                    trainable=False))
model.add(LSTM(32, return_sequences=True, dropout=0.5, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
model.add(GlobalMaxPooling1D())
model.add(Dense(10,activation='relu', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))) 
model.add(layers.Dense(2, activation="softmax"))

model.compile(optimizer=adam, loss="binary_crossentropy", 
     metrics=['accuracy'])

epochs = 25
batch_size = 64

	
es = EarlyStopping(monitor='val_loss', mode="min", patience=3)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.3, callbacks=[es])
# history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=batch_size)


Epoch 1/25
294/294 [==============================] - 82s 271ms/step - loss: 0.3951 - accuracy: 0.9046 - val_loss: 0.2403 - val_accuracy: 0.9490
Epoch 2/25
294/294 [==============================] - 78s 267ms/step - loss: 0.2264 - accuracy: 0.9471 - val_loss: 0.1947 - val_accuracy: 0.9575
Epoch 3/25
294/294 [==============================] - 78s 264ms/step - loss: 0.1996 - accuracy: 0.9508 - val_loss: 0.1792 - val_accuracy: 0.9571
Epoch 4/25
294/294 [==============================] - 78s 264ms/step - loss: 0.1827 - accuracy: 0.9539 - val_loss: 0.1751 - val_accuracy: 0.9563
Epoch 5/25
294/294 [==============================] - 78s 264ms/step - loss: 0.1729 - accuracy: 0.9541 - val_loss: 0.1748 - val_accuracy: 0.9535
Epoch 6/25
294/294 [==============================] - 78s 265ms/step - loss: 0.1657 - accuracy: 0.9555 - val_loss: 0.1562 - val_accuracy: 0.9602
Epoch 7/25
294/294 [==============================] - 78s 265ms/step - loss: 0.1603 - accuracy: 0.9563 - val_loss: 0.1566 - val_ac

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

predictions = model.predict(X_test)
predictions = [np.argmax(prediction) for prediction in predictions]
precision, recall, f1_score, none = precision_recall_fscore_support(Y_test_arr, predictions, average='weighted')
print("Precision is: ", precision)
print("Recall is: ", recall)
print("F1 Score is: ", f1_score)

360/360 [==============================] - 45s 124ms/step - loss: 0.1581 - accuracy: 0.9536
Test set
  Loss: 0.158
  Accuracy: 0.954
Precision is:  0.955613322188781
Recall is:  0.9536015292379877
F1 Score is:  0.9536323512888436


# LSTM + Skip-Gram

- Choose where you want to save the embedding_word2vec txt files.
- You will be retrieving from that directory in the next code!

In [ ]:
"""
Skip-Gram Model
"""
text_sentences = df['text'].apply(lambda x: x.split())

model = gensim.models.Word2Vec(sentences=text_sentences, size=EMBEDDING_DIM, window=5, workers=4, min_count=1, sg=1)
words = list(model.wv.vocab)
print(len(words))

# Where you want to save.
filename = 'depression_embedding_word2vec_skipgram.txt'
model.wv.save_word2vec_format(filename, binary=False)


53756


In [ ]:
# SkipGram word2vec
import numpy as np
embeddings_index = {}
f = open("/content/depression_embedding_word2vec_skipgram.txt")
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddings_index[word] = coefs
f.close()

num_words = len(word_index) + 1
word2vec_embedding_matrix = np.zeros((num_words, 128))

for word, i in word_index.items():
  if i > num_words: 
    continue

  embedding_vector = embeddings_index.get(word) 
  if embedding_vector is not None:
    word2vec_embedding_matrix[i] = embedding_vector 
print(num_words)


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)

word_index = tokenizer.word_index
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)


Y = pd.get_dummies(df['depression']).values
print('Shape of label tensor:', Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 99)
# X_train, X_test, Y_train, Y_test = train_test_split(X_svd, Y, test_size = 0.30, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)



53661
Shape of label tensor: (38362, 2)
(26853, 2521) (26853, 2)
(11509, 2521) (11509, 2)


In [ ]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
import tensorflow as tf 
from tensorflow import keras
from keras.regularizers import l2


model = Sequential()
adam = tf.optimizers.Adam(learning_rate=0.001)
model.add(Embedding(num_words, 128, weights=[word2vec_embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, 
                    trainable=False))
model.add(LSTM(32, return_sequences=True, dropout=0.5, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
model.add(GlobalMaxPooling1D())
model.add(Dense(10,activation='relu', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))) 
model.add(layers.Dense(2, activation="softmax"))

model.compile(optimizer=adam, loss="binary_crossentropy", 
     metrics=['accuracy'])

epochs = 25
batch_size = 64

	
es = EarlyStopping(monitor='val_loss', mode="min", patience=3)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.3, callbacks=[es])
# history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=batch_size)


Epoch 1/25
294/294 [==============================] - 81s 266ms/step - loss: 0.4301 - accuracy: 0.8851 - val_loss: 0.3180 - val_accuracy: 0.9103
Epoch 2/25
294/294 [==============================] - 78s 264ms/step - loss: 0.2402 - accuracy: 0.9434 - val_loss: 0.2074 - val_accuracy: 0.9546
Epoch 3/25
294/294 [==============================] - 78s 264ms/step - loss: 0.2117 - accuracy: 0.9471 - val_loss: 0.1914 - val_accuracy: 0.9539
Epoch 4/25
294/294 [==============================] - 78s 264ms/step - loss: 0.1970 - accuracy: 0.9490 - val_loss: 0.1798 - val_accuracy: 0.9549
Epoch 5/25
294/294 [==============================] - 77s 263ms/step - loss: 0.1861 - accuracy: 0.9513 - val_loss: 0.1728 - val_accuracy: 0.9566
Epoch 6/25
294/294 [==============================] - 77s 263ms/step - loss: 0.1782 - accuracy: 0.9543 - val_loss: 0.1644 - val_accuracy: 0.9590
Epoch 7/25
294/294 [==============================] - 77s 263ms/step - loss: 0.1742 - accuracy: 0.9518 - val_loss: 0.1645 - val_ac

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

predictions = model.predict(X_test)
predictions = [np.argmax(prediction) for prediction in predictions]
precision, recall, f1_score, none = precision_recall_fscore_support(Y_test_arr, predictions, average='weighted')
print("Precision is: ", precision)
print("Recall is: ", recall)
print("F1 Score is: ", f1_score)

360/360 [==============================] - 48s 134ms/step - loss: 0.1384 - accuracy: 0.9610
Test set
  Loss: 0.138
  Accuracy: 0.961
Precision is:  0.9618126254414924
Recall is:  0.9609870536102181
F1 Score is:  0.9610126628791377


# LSTM + Glove

- glove.6B.300d.txt is attached in the submission folder. Please use the appropriate directory you stored it in.

In [ ]:
# Glove
import numpy as np
embeddings_index = {}

# Use your own directory.
f = open("/content/drive/MyDrive/Colab Notebooks/IS424 Depression Project/Classification/glove.6B.300d.txt")
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddings_index[word] = coefs
f.close()

num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 300))

for word, i in word_index.items():
  if i > num_words: 
    continue

  embedding_vector = embeddings_index.get(word) 
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector 
print(num_words)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)

word_index = tokenizer.word_index
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

Y = pd.get_dummies(df['depression']).values
print('Shape of label tensor:', Y.shape)

53661
Shape of label tensor: (38362, 2)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 99)
print(X_train.shape, Y_train.shape)
print(X_test.shape,Y_test.shape)

(26853, 2521) (26853, 2)
(11509, 2521) (11509, 2)


In [ ]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
import tensorflow as tf 
from tensorflow import keras
from keras.regularizers import l2

model = Sequential()
adam = tf.optimizers.Adam(learning_rate=0.001)
model.add(Embedding(num_words, 300, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, 
                    trainable=False))
model.add(LSTM(32, return_sequences=True, dropout=0.5, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
model.add(GlobalMaxPooling1D())
model.add(Dense(10,activation='relu', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))) 
model.add(layers.Dense(2, activation="softmax"))

model.compile(optimizer=adam, loss="binary_crossentropy", 
     metrics=['accuracy'])

epochs = 25
batch_size = 64

	
es = EarlyStopping(monitor='val_loss', mode="min", patience=3)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.3, callbacks=[es])
# history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=batch_size)



Epoch 1/25
294/294 [==============================] - 85s 281ms/step - loss: 0.4398 - accuracy: 0.8854 - val_loss: 0.2614 - val_accuracy: 0.9393
Epoch 2/25
294/294 [==============================] - 82s 279ms/step - loss: 0.2492 - accuracy: 0.9387 - val_loss: 0.2128 - val_accuracy: 0.9518
Epoch 3/25
294/294 [==============================] - 82s 278ms/step - loss: 0.2229 - accuracy: 0.9438 - val_loss: 0.2097 - val_accuracy: 0.9466
Epoch 4/25
294/294 [==============================] - 82s 278ms/step - loss: 0.2045 - accuracy: 0.9481 - val_loss: 0.1868 - val_accuracy: 0.9541
Epoch 5/25
294/294 [==============================] - 82s 278ms/step - loss: 0.1997 - accuracy: 0.9474 - val_loss: 0.1836 - val_accuracy: 0.9552
Epoch 6/25
294/294 [==============================] - 82s 278ms/step - loss: 0.1868 - accuracy: 0.9508 - val_loss: 0.1770 - val_accuracy: 0.9553
Epoch 7/25
294/294 [==============================] - 82s 278ms/step - loss: 0.1802 - accuracy: 0.9540 - val_loss: 0.1733 - val_ac

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

predictions = model.predict(X_test)
predictions = [np.argmax(prediction) for prediction in predictions]
precision, recall, f1_score, none = precision_recall_fscore_support(Y_test_arr, predictions, average='weighted')
print("Precision is: ", precision)
print("Recall is: ", recall)
print("F1 Score is: ", f1_score)

360/360 [==============================] - 34s 94ms/step - loss: 0.1568 - accuracy: 0.9598
Test set
  Loss: 0.157
  Accuracy: 0.960
Precision is:  0.9601075235415188
Recall is:  0.9597706143018507
F1 Score is:  0.9597907993001087


# LSTM + Pre-trained Word2Vec

In [ ]:
"""
Pre-trained Word2Vec Model
This part loads really long, please be patient!
"""

import gensim.downloader as api
model = api.load("word2vec-google-news-300")


In [ ]:
"""
Pre-trained Word2Vec Model
"""

text_sentences = df['text'].apply(lambda x: x.split())
words = list(model.wv.vocab)
print(len(words))

# filename = 'selfdriving_embedding_word2vec_pretrained.txt'
# model.wv.save_word2vec_format(filename, binary=False)

In [ ]:
import numpy as np
labels = np.asarray(model.index2word)
vectors = np.asarray(model.vectors)
word_embeddings = dict(zip(labels, vectors))

num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 300))

for word, i in word_index.items():
  if i > num_words: 
    continue

  embedding_vector = word_embeddings.get(word) 
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector 
print(num_words)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)

word_index = tokenizer.word_index
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

Y = pd.get_dummies(df['depression']).values
print('Shape of label tensor:', Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 99)
# X_train, X_test, Y_train, Y_test = train_test_split(X_svd, Y, test_size = 0.30, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

53661
Shape of label tensor: (38362, 2)
(26853, 2521) (26853, 2)
(11509, 2521) (11509, 2)


In [ ]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
import tensorflow as tf 
from tensorflow import keras
from keras.regularizers import l2

model = Sequential()
adam = tf.optimizers.Adam(learning_rate=0.001)
model.add(Embedding(num_words, 300, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, 
                    trainable=False))
model.add(LSTM(32, return_sequences=True, dropout=0.5, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
model.add(GlobalMaxPooling1D())
model.add(Dense(10,activation='relu', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))) 
model.add(layers.Dense(2, activation="softmax"))

model.compile(optimizer=adam, loss="binary_crossentropy", 
     metrics=['accuracy'])

epochs = 25
batch_size = 64

	
es = EarlyStopping(monitor='val_loss', mode="min", patience=3)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.3, callbacks=[es])
# history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=batch_size)


Epoch 1/25
294/294 [==============================] - 89s 280ms/step - loss: 0.4425 - accuracy: 0.9022 - val_loss: 0.2597 - val_accuracy: 0.9458
Epoch 2/25
294/294 [==============================] - 81s 276ms/step - loss: 0.2574 - accuracy: 0.9402 - val_loss: 0.2242 - val_accuracy: 0.9525
Epoch 3/25
294/294 [==============================] - 81s 276ms/step - loss: 0.2306 - accuracy: 0.9464 - val_loss: 0.2024 - val_accuracy: 0.9526
Epoch 4/25
294/294 [==============================] - 81s 276ms/step - loss: 0.2167 - accuracy: 0.9467 - val_loss: 0.1883 - val_accuracy: 0.9556
Epoch 5/25
294/294 [==============================] - 81s 276ms/step - loss: 0.2049 - accuracy: 0.9483 - val_loss: 0.1808 - val_accuracy: 0.9561
Epoch 6/25
294/294 [==============================] - 81s 276ms/step - loss: 0.1960 - accuracy: 0.9505 - val_loss: 0.1837 - val_accuracy: 0.9572
Epoch 7/25
294/294 [==============================] - 81s 275ms/step - loss: 0.1887 - accuracy: 0.9515 - val_loss: 0.1807 - val_ac

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

predictions = model.predict(X_test)
predictions = [np.argmax(prediction) for prediction in predictions]
precision, recall, f1_score, none = precision_recall_fscore_support(Y_test_arr, predictions, average='weighted')
print("Precision is: ", precision)
print("Recall is: ", recall)
print("F1 Score is: ", f1_score)

360/360 [==============================] - 37s 103ms/step - loss: 0.1426 - accuracy: 0.9632
Test set
  Loss: 0.143
  Accuracy: 0.963
Precision is:  0.9632444777731072
Recall is:  0.9632461551829004
F1 Score is:  0.9632449591810578


# Bi-LSTM + Glove
- glove.6B.300d.txt is attached in the submission folder. Please use the appropriate directory you stored it in.

In [ ]:
# Glove
import numpy as np
embeddings_index = {}


f = open("/content/drive/MyDrive/Colab Notebooks/IS424 Depression Project/Finalised Submission Folder/glove.6B.300d.txt")
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddings_index[word] = coefs
f.close()

num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 300))

for word, i in word_index.items():
  if i > num_words: 
    continue

  embedding_vector = embeddings_index.get(word) 
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector 
print(num_words)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)

word_index = tokenizer.word_index
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

Y = pd.get_dummies(df['depression']).values
print('Shape of label tensor:', Y.shape)


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 99)
# X_train, X_test, Y_train, Y_test = train_test_split(X_svd, Y, test_size = 0.30, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)



53661
Shape of label tensor: (38362, 2)
(26853, 2521) (26853, 2)
(11509, 2521) (11509, 2)


In [ ]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
import tensorflow as tf 
from tensorflow import keras
from keras.regularizers import l2

model = Sequential()
adam = tf.optimizers.Adam(learning_rate=0.001)
model.add(Embedding(num_words, 300, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, 
                    trainable=False))
model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=0.5, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001))))
model.add(GlobalMaxPooling1D())
model.add(Dense(10,activation='relu', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))) 
model.add(layers.Dense(2, activation="softmax"))

model.compile(optimizer=adam, loss="binary_crossentropy", 
     metrics=['accuracy'])

epochs = 25
batch_size = 64

	
es = EarlyStopping(monitor='val_loss', mode="min", patience=3)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.3, callbacks=[es])
# history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=batch_size)


Epoch 1/25
294/294 [==============================] - 175s 566ms/step - loss: 0.5291 - accuracy: 0.8786 - val_loss: 0.2843 - val_accuracy: 0.9464
Epoch 2/25
294/294 [==============================] - 165s 563ms/step - loss: 0.2657 - accuracy: 0.9427 - val_loss: 0.2293 - val_accuracy: 0.9516
Epoch 3/25
294/294 [==============================] - 165s 563ms/step - loss: 0.2286 - accuracy: 0.9483 - val_loss: 0.2082 - val_accuracy: 0.9525
Epoch 4/25
294/294 [==============================] - 165s 562ms/step - loss: 0.2173 - accuracy: 0.9461 - val_loss: 0.1984 - val_accuracy: 0.9544
Epoch 5/25
294/294 [==============================] - 165s 562ms/step - loss: 0.1967 - accuracy: 0.9530 - val_loss: 0.1919 - val_accuracy: 0.9530
Epoch 6/25
294/294 [==============================] - 166s 566ms/step - loss: 0.1894 - accuracy: 0.9525 - val_loss: 0.1798 - val_accuracy: 0.9559
Epoch 7/25
294/294 [==============================] - 167s 568ms/step - loss: 0.1813 - accuracy: 0.9545 - val_loss: 0.1728 -

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

predictions = model.predict(X_test)
predictions = [np.argmax(prediction) for prediction in predictions]
precision, recall, f1_score, none = precision_recall_fscore_support(Y_test_arr, predictions, average='weighted')
print("Precision is: ", precision)
print("Recall is: ", recall)
print("F1 Score is: ", f1_score)

360/360 [==============================] - 67s 186ms/step - loss: 0.1572 - accuracy: 0.9614
Test set
  Loss: 0.157
  Accuracy: 0.961
Precision is:  0.9614523505560858
Recall is:  0.9614214962203493
F1 Score is:  0.9614274979874109
